# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [94]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [95]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [96]:
# #Dados de autenticação do twitter:

# #Coloque aqui o identificador da conta no twitter: @fulano

# #leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

# #Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [97]:
# #Produto escolhido:
# produto = 'dota'

# #Quantidade mínima de mensagens capturadas:
# n = 500
# #Quantidade mínima de mensagens para a base de treinamento:
# t = 300

# #Filtro de língua, escolha uma na tabela ISO 639-1.
# lang = 'en'

Capturando os dados do twitter:

In [98]:
# #Cria um objeto para a captura
# api = tweepy.API(auth)

# #Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# msgs = []
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
#     msgs.append(msg.text.lower())
#     i += 1
#     if i > n:
#         break

# #Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [99]:
# #Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
#     #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

#     #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

#     #fecha o arquivo
#     writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [100]:
dfdota=pd.read_excel('Dota.xlsx', sep=',')
#dfdota

In [101]:
dfdota['Treinamento']=dfdota.Treinamento.str.replace(':','')
dfdota['Treinamento']=dfdota.Treinamento.str.replace('"','')
dfdota['Treinamento']=dfdota.Treinamento.str.replace("'","")
dfdota['Treinamento']=dfdota.Treinamento.str.replace('(','')
dfdota['Treinamento']=dfdota.Treinamento.str.replace(')','')
dfdota['Treinamento']=dfdota.Treinamento.str.replace('\n','')
dfdota['Treinamento']=dfdota.Treinamento.str.replace(',','')
dfdota['Treinamento']=dfdota.Treinamento.str.replace('.','')
#dfdota['Treinamento']=dfdota.Treinamento.str.replace('@','')
dfdota['Treinamento'].str.lower()
dfdota

,Treinamento,Classificacao,Unnamed: 2,Legenda
0,@unxarted joga dota 2 comigo cc,1,NaN,irrelevante=0
1,preciso dormir preciso estudar ai eu vou e vir...,1,NaN,relevante=1
2,@levieholanda viado dota é muito mais evoluído...,1,NaN,NaN
3,@fiaspo eu to jogando dota homem,0,NaN,NaN
4,come dota,1,NaN,NaN
5,eu q sou -77% nerd peguei a editoria mais nerd...,1,NaN,NaN
6,quit dota,1,NaN,NaN
7,prefiro dota doq lol,0,NaN,NaN
8,@luanhf_ dota quarta pra quinta vem e se vira ...,1,NaN,NaN
9,se no novo patch do dota não neffarem a legion...,0,NaN,NaN


In [102]:
dfdota.to_csv('var_dota.csv')

In [103]:
dfdota['Treinamento']=dfdota['Treinamento'].str.split(pat=None,n=-1,expand=False)
dfdota

,Treinamento,Classificacao,Unnamed: 2,Legenda
0,"[@unxarted, joga, dota, 2, comigo, cc]",1,NaN,irrelevante=0
1,"[preciso, dormir, preciso, estudar, ai, eu, vo...",1,NaN,relevante=1
2,"[@levieholanda, viado, dota, é, muito, mais, e...",1,NaN,NaN
3,"[@fiaspo, eu, to, jogando, dota, homem]",0,NaN,NaN
4,"[come, dota]",1,NaN,NaN
5,"[eu, q, sou, -77%, nerd, peguei, a, editoria, ...",1,NaN,NaN
6,"[quit, dota]",1,NaN,NaN
7,"[prefiro, dota, doq, lol]",0,NaN,NaN
8,"[@luanhf_, dota, quarta, pra, quinta, vem, e, ...",1,NaN,NaN
9,"[se, no, novo, patch, do, dota, não, neffarem,...",0,NaN,NaN


In [108]:
# Calculando probabilidade total
dfdotaT=dfdota['Treinamento'].sum(axis=None, skipna=None, level=None, numeric_only=None)
#dfdotaT
serie = pd.Series(dfdotaT)
serie_vc = serie.value_counts()
#Pt= probabilidade total de cada palavra no df
Pt=(serie_vc+1)/(len(serie) + len(serie_vc))

In [109]:
serie1 = pd.Series(dfdota['Classificacao'])
serie_vc1 = serie1.value_counts()
#Pf= probabilidade de relevantcia total no df
Pf=(serie_vc1+1)/(len(serie1) + len(serie_vc1))

In [120]:
#FILTRANDO DF RELEVANTE
dfdota1=dfdota[dfdota.Classificacao==1]#.Treinamento
dfdota1=dfdota1['Treinamento'].sum(axis=None, skipna=None, level=None, numeric_only=None)
serie1 = pd.Series(dfdota1)
serie_vc1 = serie1.value_counts()
#Pr= probabilidade  de cada palavra relevante no df
Pr=(serie_vc1+1)/(len(serie1) + len(serie_vc1))
Pr

dota                     0.055636
de                       0.030237
e                        0.016449
eu                       0.014998
um                       0.013304
jogar                    0.012337
2                        0.011853
é                        0.011369
a                        0.010160
que                      0.009918
no                       0.008708
o                        0.008466
@youtube                 0.007983
-                        0.007983
do                       0.007741
vídeo                    0.006773
gostei                   0.006773
com                      0.006289
não                      0.005806
q                        0.005564
lol                      0.005564
da                       0.005322
jogo                     0.005080
pra                      0.005080
joga                     0.004596
to                       0.004354
me                       0.004354
uma                      0.004112
tem                      0.003870
só            

In [119]:
#FILTRANDO DF IRRELEVANTE
dfdota0=dfdota[dfdota.Classificacao==0]#.Treinamento
dfdota0=dfdota0['Treinamento'].sum(axis=None, skipna=None, level=None, numeric_only=None)
serie0 = pd.Series(dfdota0)
serie_vc0 = serie0.value_counts()
#Pi= probabilidade  de cada palavra irrelevante no df
Pi=(serie_vc0+1)/(len(serie0) + len(serie_vc0))

In [ ]:
# Probabilidade da palavra se encaixar na categoria relevante
#palavra ='dota'
# Teste
#Ppr = (Pr[palavra]*Pf[1])/Pt[palavra]
#Ppi = (Pi[palavra]*Pf[0])/Pt[palavra]
#Ppr
#Pr


In [123]:
newR=[]
newI=[]

for i in dfdotaT:
    if i not in Pr:
        Ppi = (Pi[i]*Pf[0])/Pt[i]
        Ppr=1-Ppi
        newR.append(Ppr)
        newI.append(Ppi)
    elif i not in Pi:
        Ppr = (Pr[i]*Pf[1])/Pt[i]
        Ppi=1-Ppr
        newR.append(Ppr)
        newI.append(Ppi)
    else:
        Ppr = (Pr[i]*Pf[1])/Pt[i]
        Ppi = (Pi[i]*Pf[0])/Pt[i]
        newR.append(Ppr)
        newI.append(Ppi)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
